# Python Client

In [ ]:
# Import ModelBazaar class.
import thirdai
from thirdai.neural_db import ModelBazaar

# Activate your license
thirdai.licensing.activate("YOUR-THIRDAI-ACTIVATION-KEY")

In [ ]:
# Initialise a ModelBazaar object with url where model_bazaar is hosted.
# Append `/api/` in the url.
bazaar = ModelBazaar(base_url="http://20.29.78.64//api/")

In [ ]:
# Used to signup on model bazaar. Sends a email verification link.
bazaar.sign_up(email="mritunjay@thirdai.com", password="password", username="mj3ai")

In [ ]:
# Login with your email_id and password.
bazaar.log_in(email="mritunjay@thirdai.com", password="password")

## Data generation

In [ ]:
text_form_data = {
    "samples_per_label": 2,
    "target_labels": ["toxic", "non-toxic"],
    "examples": {"toxic": ["You are stupid, ugly person"], "non-toxic": ["You've been invited to the talk show"]},
    "labels_description": {"toxic": "Derogatory text, comments", "not-spam": "normal text, conversational text"},
    "batch_size": 10,
    "vocab_per_sentence": 2
}

    
token_form_data = {
    "domain_prompt": "Personal and financial information for a person",
    "tags": ['O','CARDNUMBER','NAME','CVV'],
    "tag_examples": {"O": ["Third", "Party"],"CARDNUMBER": ["4589-6895-7848", "1487-8578-1839-7818", "981571871382"],"NAME": ["Larson", "Dakota"],"CVV": ["458", "041"]},
    "num_call_batches": 2,
    "batch_size": 5,
    "num_samples_per_tag": 4
}

In [ ]:
bazaar.generate_data(
    task_prompt = "Ner model for tagging personal and financial information for a person",
    data_cateogry="token",
    form_data = token_form_data
)

## Training model on document

In [ ]:
train_extra_options = {
    "num_samples_per_shard": 3_500_000,
    "allocation_memory": 300000,
    #   ---shard agnostic training params---
    "model_cores": 20,
    "model_memory": 200000,
    "csv_id_column": "label",
    "csv_strong_columns": ["title"],
    "csv_weak_columns": ["abstract"],
    "csv_reference_columns": ["title", "abstract"],
    "fhr": 200000,
    "embedding_dim": 4096,
    "output_dim": 100000,
    "max_in_memory_batches": 2000,
}

In [ ]:
# Creates a model from scratch trained on given list of documents.
# Can be synchronous or asynchronous(default; in which case we call await_train)
# `doc_type` can be "local"(default), "nfs" or "s3".
model = bazaar.train(
    model_name="pubmed-35M-10-models-char4-0",
    docs=["/model_bazaar/datasets/pubmed/lowercased_all_withoutpapertags.csv"],
    doc_type="nfs",
    sharded=True,
    is_async=True,
    train_extra_options=train_extra_options,
)

In [ ]:
# Blocking call to wait till model finishes training.
bazaar.await_train(model)

In [ ]:
# Deploys the model and returns an ndb_client which can be used to interact to neural_db.
# Can be synchronous or asynchronous(default; in which case we call await_deploy)
ndb_client = bazaar.deploy(
    model_identifier=model.model_identifier,
    deployment_name="deployment-0",
    is_async=True,
)

In [ ]:
# Blocking call to wait till deployment finishes.
bazaar.await_deploy(ndb_client)

In [ ]:
# Insert new files in the neural_db model.
ndb_client.insert(
    files=["/Users/mjay/Documents/MACH.pdf", "/Users/mjay/Documents/OpenMPIInstall.pdf"]
)

In [ ]:
# Search the ndb model.
results = ndb_client.search(query="who are the authors of this paper", top_k="5")

query_text = results["query_text"]
references = results["references"]
for reference in references:
    print(reference["text"])

In [ ]:
# [RLHF] Associate : takes list of dictionaries where each dictionary has 'source' and 'target' keys.
ndb_client.associate(
    [
        {"source": "authors", "target": "contributors"},
        {"source": "paper", "target": "document"},
    ]
)

In [ ]:
# [RLHF] Upvote/Downvote : takes list of dictionaries where each dictionary has 'query_text' and 'reference_id' keys.
best_answer = references[4]
good_answer = references[2]
ndb_client.upvote(
    [
        {"query_text": query_text, "reference_id": best_answer["id"]},
        {"query_text": query_text, "reference_id": good_answer["id"]},
    ]
)

In [ ]:
# Deletes documents from the deployment so they are not able to be searched over anymore
ndb_client.delete([references[0]["source_id"]])

In [ ]:
# Undeploys the model.
bazaar.undeploy(ndb_client)

In [ ]:
# Deletes the model from model bazaar.
bazaar.delete(model_identifier=model.model_identifier)

In [ ]:
# Return a list of models accessible to user on model bazaar.
bazaar.list_models()

In [ ]:
# Returns a list of all active deployments.
bazaar.list_deployments()

In [ ]:
# Connects to an active deployment.
ndb_client = bazaar.connect(deployment_identifier="mj3ai/model-0:mj3ai/deployment-0")

In [ ]:
# Push an NDB model from local to model bazaar
bazaar.push_model(
    model_name="test-upload-2",
    local_path="/Users/mjay/test.ndb",
    access_level="private",
)

In [ ]:
# Pull model from model bazaar to local
ndb_model = bazaar.pull_model(model_identifier="mj3ai/model-1")